#### 임폴트

In [5]:
import os
import json
import time
import re
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableLambda

#### 데이터 로드 및 준비

In [12]:
# JSON 파일 로드
file_path = "./duplicates.json"

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터프레임 변환
df = pd.DataFrame(data)
df = df[['title', 'link', 'date', 'content', 'source']]
df["date"] = pd.to_datetime(df["date"])
df['quarter'] = df["date"].dt.to_period("Q")

# 광고 및 출처 정보 제거 정규식 패턴
AD_PATTERNS = [
    r"责任编辑：.*?",
    r"（本文来源.*?）",
    r"点击阅读全文.*?",
    r"来源：.*?",
    r"更多精彩内容.*?",
    r"本文转载自.*?",
    r"欢迎关注我们的.*?",
    r"广告.*?",
    r"（.*?记者.*?报道）",
    r"如需转载请注明.*?",
    r"查看更多相关信息.*?"
]

def clean_text(text):
    """불필요한 광고 및 출처 제거하고, 구두점 유지"""
    if not isinstance(text, str):
        return ""

    # 광고 및 출처 제거
    for pattern in AD_PATTERNS:
        text = re.sub(pattern, "", text)

    # 특수 문자 및 공백 정리 (단, 구두점 `。！？`는 유지)
    text = re.sub(r'[^\w\s。！？]', '', text)  # 한자, 숫자, 구두점 외 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 정리

    return text

df['input_text'] = df['content'].apply(clean_text)

#### llm 모델설정

In [13]:
# 환경 변수 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0)

# # 메모리 설정 (대화 컨텍스트 유지)
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#
# # 규칙을 메모리에 직접 저장
# memory.save_context(
#     {"input_text": "이제부터 감성 분석 규칙을 적용합니다."},
#     {"response": "1. 객관적 사실 전달만 포함된 기사는 0점.\n"
#                  "2. 긍정과 부정이 혼재된 경우, 감성 강도에 따라 가중평균을 적용하여 점수를 산정.\n"
#                  "3. 감성 점수만 숫자로 출력하고, 설명은 포함하지 않음."}
# )
#
# def memory_loader_func(input_data):
#     """메모리에서 `chat_history`를 추가하면서 `input_text`를 유지"""
#     chat_history = memory.load_memory_variables({})
#     chat_history["input_text"] = input_data["input_text"]  # ✅ input_text 유지
#     return chat_history
#
# # 메모리 불러오기
# memory_loader = RunnableLambda(memory_loader_func)

# 감성 분석 프롬프트 (규칙 포함)
basic_prompt_template = PromptTemplate(
    input_variables=["input_text"],
    template=(
        "다음 기사 본문의 감성을 반드시 **-5에서 +5사이**의 정수로 평가하세요.\n"
        "규칙:\n"
        "1. 객관적 사실 전달만 포함된 기사는 0점.\n"
        "2. 긍정과 부정이 혼재된 경우, 0점을 주지말고 최대한 채점해주세요.\n"
        "3. 반드시 숫자만 출력하고, 설명이나 텍스트를 포함하지 마세요.\n"
        "기사 본문: {input_text}\n"
        "답변:"
    )
)

# 기본 프롬프트 체인
sentiment_chain = basic_prompt_template | llm

In [8]:
# ✅ 감성 분석 실행 함수
def analyze_sentiment(text, max_retries=1):
    """기사 본문 전체를 감성 분석 (모든 요청에 프롬프트 포함)"""
    for attempt in range(max_retries):
        try:
            # ✅ 항상 프롬프트 포함하여 OpenAI가 규칙을 유지
            formatted_prompt = basic_prompt_template.format(input_text=text)

            response = llm.invoke(formatted_prompt)  # ✅ invoke()에 문자열 직접 전달
            result_text = response.content.strip()

            # ✅ 숫자만 추출 (감성 점수가 아닌 텍스트가 반환될 경우 예외 처리)
            match = re.search(r"-?\d+", result_text)
            if match:
                score = int(match.group())

                # ✅ -5 ~ +5 범위 확인 (초과하는 값은 None 반환)
                if -5 <= score <= 5:
                    return score
                else:
                    print(f"⚠️ 범위를 벗어난 감성 점수 ({score}) → 무효 처리")
                    return None
            else:
                print(f"⚠️ 숫자 감성 점수 추출 실패: {result_text}")
                return None  # 숫자가 없으면 None 반환

        except Exception as e:
            error_msg = str(e)
            print(f"⚠️ 오류 발생: {error_msg}, 재시도 {attempt + 1}/{max_retries}")

            # ✅ 429 Rate Limit 오류 감지 (자동 대기 후 재시도)
            if "rate_limit_exceeded" in error_msg:
                try:
                    error_json = json.loads(error_msg.split(" - ")[1])
                    wait_time = float(error_json['error']['message'].split("Please try again in ")[1].split("s")[0])
                    print(f"⏳ {wait_time}초 후 재시도...")
                    time.sleep(wait_time + 1)
                except:
                    time.sleep(5)  # 기본 5초 대기
            else:
                time.sleep(3)  # 일반 오류 시 3초 대기

    return None

In [14]:
backup_file = "sentiment_analysis_2.csv"
# 기존 df가 이미 존재하는 경우, 백업 파일이 없으면 저장
if not os.path.exists(backup_file):
    df.to_csv(backup_file, index=False)
    print(f"✅ 백업 파일이 없어서 새로 저장함: {backup_file}")
else:
    print("✅ 기존 백업 파일이 존재함.")

# 'sentiment_score' 컬럼이 없으면 추가
if "sentiment_score" not in df.columns:
    df["sentiment_score"] = None
    print("✅ 'sentiment_score' 컬럼 추가함.")

# 감성 분석이 수행되지 않은 행 찾기
incomplete_rows = df[df["sentiment_score"].isna()]

# 병렬 처리 실행
def process_row(index, text):
    score = analyze_sentiment(text)
    df.at[index, "sentiment_score"] = score  # 결과 업데이트
    time.sleep(3)  # 각 요청마다 3초 대기 추가 (429 방지)
    return index

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(process_row, idx, row["input_text"]): idx for idx, row in incomplete_rows.iterrows()}

    for i, future in enumerate(futures):
        future.result()  # 실행 완료 대기
        if (i + 1) % 5 == 0:  # 5개 완료될 때마다 저장
            df.to_csv(backup_file, index=False)
            print(f"🔄 {i + 1}개 처리 완료, 데이터 저장!")

# 마지막 데이터 저장
df.to_csv(backup_file, index=False)
print("✅ 모든 데이터 처리 완료 및 저장됨!")

✅ 백업 파일이 없어서 새로 저장함: sentiment_analysis_2.csv
✅ 'sentiment_score' 컬럼 추가함.
🔄 5개 처리 완료, 데이터 저장!
🔄 10개 처리 완료, 데이터 저장!
🔄 15개 처리 완료, 데이터 저장!
🔄 20개 처리 완료, 데이터 저장!
🔄 25개 처리 완료, 데이터 저장!
🔄 30개 처리 완료, 데이터 저장!
🔄 35개 처리 완료, 데이터 저장!
🔄 40개 처리 완료, 데이터 저장!
🔄 45개 처리 완료, 데이터 저장!
🔄 50개 처리 완료, 데이터 저장!
🔄 55개 처리 완료, 데이터 저장!
🔄 60개 처리 완료, 데이터 저장!
🔄 65개 처리 완료, 데이터 저장!
🔄 70개 처리 완료, 데이터 저장!
⚠️ 오류 발생: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-A1JNNlkMXNItEVSFqRa3W3Ch on tokens per min (TPM): Limit 30000, Used 28481, Requested 2018. Please try again in 998ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}, 재시도 1/1
🔄 75개 처리 완료, 데이터 저장!
🔄 80개 처리 완료, 데이터 저장!
⚠️ 오류 발생: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-A1JNNlkMXNItEVSFqRa3W3Ch on tokens per min (TPM): Limit 30000, Used 28436, Requested 2208. Please tr

In [9]:
# df 상위 10개 데이터만 사용
test_df = df.head(3).copy()

# ✅ 'sentiment_score' 컬럼 추가 (없을 경우)
if "sentiment_score" not in test_df.columns:
    test_df["sentiment_score"] = None
    print("✅ 'sentiment_score' 컬럼 추가함.")

# ✅ 감성 분석이 수행되지 않은 행 찾기
incomplete_rows = test_df[test_df["sentiment_score"].isna()]

# ✅ 병렬 처리 실행
def process_row(index, text):
    score = analyze_sentiment(text)
    test_df.at[index, "sentiment_score"] = score if score is not None else float('nan')  # ✅ NaN 처리
    time.sleep(3)  # ✅ 각 요청마다 3초 대기 추가 (429 방지)
    return index

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(process_row, idx, row["input_text"]): idx for idx, row in incomplete_rows.iterrows()}

    for i, future in enumerate(futures):
        future.result()  # 실행 완료 대기
        if (i + 1) % 5 == 0:  # 5개 완료될 때마다 출력
            print(f"🔄 {i + 1}개 처리 완료")

# ✅ 감성 분석 결과 저장
test_file = "./sentiment_analysis_test2.csv"
test_df.to_csv(test_file, index=False)
print(f"✅ 테스트 데이터 감성 분석 완료! 결과 저장: {test_file}")

✅ 'sentiment_score' 컬럼 추가함.
✅ 테스트 데이터 감성 분석 완료! 결과 저장: ./sentiment_analysis_test2.csv
